`````{note}
This lecture is going to:
* Review what we mean by features in machine learning models
* Demonstrate polynomial fits for a noisy test function with known structure
* Formalize our idea of under/over fitting and show how model capacity influences train and validation errors
* Introduce regularization as an approach to reduce the train/validation gap and overfitting in models with many features
`````

`````{seealso}
* https://scikit-learn.org/stable/modules/feature_selection.html
`````

# Model Capacity, Overfitting, Regularization, and Ridge/LASSO

We're now at the point where we may have many features in a dataset, or where we want to generate many features to build predictive models. We need ways to generalize this process, and control how many features are used. 

As a reminder, we're currently interested in solving supervised regression problems of the form:
\begin{align*}
\hat{y}=f(\mathbf{X})
\end{align*}
where 
* $\hat{y}$ is a vector of outputs (one per data point)
* $\mathbf{X}$ is a 2D array of features (one row per data point, one column per feature)

We're only going to use linear regression models in today's lecture.

## Test function (Himmelblau's function)

Let's start with the special function we used for the optimization lecture
\begin{align}
f(x, y) = (x^2+y-11)^2+(x+y^2-7)^2
\end{align}
For clarity, I'm going to write this as 
\begin{align}
y=f(x_0, x_1) &= (x_0^2+x_2-11)^2+(x_0+x_1^2-7)^2\\
&=x_0^4+x_1^4+2x_0x_1^2+2x_1x_0^2-21x_0^2-13x_1^2-14x_0-22x_1+170
\end{align}
where y is the output/label, and $x_0,x_1$ are potential features (among many that we could choose)! This is a nice test function since it's analytical and is polynomial. If we try to find polynomial features of $x_0,x_1$, we should recover this form. 

```{seealso}
https://en.wikipedia.org/wiki/Himmelblau%27s_function
```

First, let's define the range of $x_1,x_2$ values we're interested in plotting over, and turn them into a 2D grid like we did in the optimization lecture.

In [ ]:
import numpy as np

x1range = np.linspace(-5, 5)
x2range = np.linspace(-5, 5)

# Make 2d arrays for all the unique values of x_1/x_2
x1grid, x2grid = np.meshgrid(x1range, x2range)

Now, let's define a function to return Himmelblau's function, plus a little bit of random noise

In [ ]:
import numpy as np


def himmelblau_with_noise(x1, x2, noise=0.1, seed=42):

    # Set the numpy random seed so the results are reproducible
    np.random.seed(seed)

    # Generate the himmelblau function
    himmelblau = (x1**2 + x2 - 11) ** 2 + (x1 + x2**2 - 7) ** 2

    # Multiple by a bit of Gaussian random noise and return
    noise = np.random.normal(loc=1, scale=noise, size=x1.shape)
    return himmelblau * noise

Finally, let's plot the function without noise, and add points for 20 random points in that space. 

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Surface(
            x=x1range, y=x2range, z=himmelblau_with_noise(x1grid, x2grid, noise=0)
        )
    ]
)

# Generate 20 samples from the noisy Himmelblau function
nsamples = 20
X = np.random.uniform(low=-5, high=5, size=(nsamples, 2))
y = himmelblau_with_noise(X[:, 0], X[:, 1])

# Plot with plotly
fig.add_scatter3d(
    x=X[:, 0], y=X[:, 1], z=y, mode="markers", marker=dict(size=6, color="#00FF00")
)
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

## Data generation: Generate 60 random samples and a random 60/20/20 train/val/test split

Before we do anything, we need to separate our data into train/val/test splits so that we don't overfit and we have some idea of how predictive our model is!

In [ ]:
from sklearn.model_selection import train_test_split


# Helper function to generate train/val/test splits for
def generate_trainvaltest_data(nsamples):

    # Sample npoints random x1/x2 values
    np.random.seed(42)
    X = np.random.uniform(low=-5, high=5, size=(nsamples, 2))

    # Evaluate the function
    y = himmelblau_with_noise(X[:, 0], X[:, 1])

    # Generate the train/val/test splits
    X_trainval, X_test, y_trainval, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=True, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval, test_size=0.25, shuffle=True, random_state=42
    )

    return X_train, X_val, X_test, y_train, y_val, y_test


X_train, X_val, X_test, y_train, y_val, y_test = generate_trainvaltest_data(60)

## Model fitting

### Base case (linear model with $x_0$, $x_1$ as the only the features)

Our goal will be to find the polynomial features that make this an easy function to fit. Before we do something interesting, let's start with the simplest thing we can try: a linear function using just $x_0$ and $x_1$ as features.

We'll use scikit-learn now that we've seen an example through the homework.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)

mae = mean_absolute_error(y_val, model.predict(X_val))
rmse = mean_squared_error(y_val, model.predict(X_val)) ** 0.5

print(
    f"The MAE for the linear fit is {mae:0.2f}, compared to a standard deviation of {y_val.std():0.2f}"
)
print(
    f"The RMSE for the linear fit is {rmse:0.2f}, compared to a standard deviation of {y_val.std():0.2f}"
)

In [ ]:
def sklearn_predict_on_grid(model, x1grid, x2grid):
    # An sklearn pipeline or model will expect to predict on a NxM array,
    # where N is the number of data points and M is the number of features
    # This helper function will take two
    X = np.hstack((x1grid.reshape((-1, 1)), (x2grid.reshape((-1, 1)))))
    y = model.predict(X)

    return y.reshape(x1grid.shape)


# Test the function on the grid we were predicting on!
sklearn_predict_on_grid(model, x1grid, x2grid)

Now, let's plot the linear fit, as well as the train and validation data.

In [ ]:
import plotly.graph_objects as go

# Plot the fitted model
fig = go.Figure(
    data=[
        go.Surface(
            x=x1range, y=x2range, z=sklearn_predict_on_grid(model, x1grid, x2grid)
        )
    ]
)

# Plot the train data
fig.add_scatter3d(
    x=X_train[:, 0],
    y=X_train[:, 1],
    z=y_train,
    mode="markers",
    marker=dict(size=6, color="#00FF00"),
    name="Train Data",
)
# Plot the validation data
fig.add_scatter3d(
    x=X_val[:, 0],
    y=X_val[:, 1],
    z=y_val,
    mode="markers",
    marker=dict(size=6, color="#000000"),
    name="Validation Data",
)
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

In [ ]:
# Plot the fitted model
fig = go.Figure(data=[go.Scatter(x=y_val, y=model.predict(X_val), mode="markers")])

fig.add_shape(
    type="line",
    x0=y_val.min(),
    y0=y_val.min(),
    x1=y_val.max(),
    y1=y_val.max(),
)

fig.update_xaxes(
    title_text="Actual Value",
)
fig.update_yaxes(
    title_text="Predicted Value",
)

# Set the plot size
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

### Polynomial case (linear model with polynomial features)

You saw in homework 2 that we can ask scikit-learn to generate polynomial features as part of a pipeline. Let's use that to generate all of the features up to the fourth power to fit. We know that fourth order is the right place to stop because we know the functional form, so otherwise this would be something we play around with. 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

model = LinearRegression()
model = make_pipeline(PolynomialFeatures(4), LinearRegression(fit_intercept=False))
model.fit(X_train, y_train)

mae = mean_absolute_error(y_val, model.predict(X_val))
rmse = mean_squared_error(y_val, model.predict(X_val)) ** 0.5

print(
    f"The MAE for the polynomial fit is {mae:0.2f}, compared to a standard deviation of {y_val.std():0.2f}"
)
print(
    f"The RMSE for the polynomial fit is {rmse:0.2f}, compared to a standard deviation of {y_val.std():0.2f}"
)

In [ ]:
import plotly.graph_objects as go

# Plot the fitted model
fig = go.Figure(
    data=[
        go.Surface(
            x=x1range, y=x2range, z=sklearn_predict_on_grid(model, x1grid, x2grid)
        )
    ]
)
fig = go.Figure(
    data=[
        go.Surface(
            x=x1range, y=x2range, z=sklearn_predict_on_grid(model, x1grid, x2grid)
        )
    ]
)

# Plot the train data
fig.add_scatter3d(
    x=X_train[:, 0],
    y=X_train[:, 1],
    z=y_train,
    mode="markers",
    marker=dict(size=6, color="#00FF00"),
    name="Train Data",
)
# Plot the validation data
fig.add_scatter3d(
    x=X_val[:, 0],
    y=X_val[:, 1],
    z=y_val,
    mode="markers",
    marker=dict(size=6, color="#000000"),
    name="Validation Data",
)
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

In [ ]:
# Parity plot!
fig = go.Figure(data=[go.Scatter(x=y_val, y=model.predict(X_val), mode="markers")])

fig.add_shape(
    type="line",
    x0=y_val.min(),
    y0=y_val.min(),
    x1=y_val.max(),
    y1=y_val.max(),
)

fig.update_xaxes(
    title_text="Actual Value",
)
fig.update_yaxes(
    title_text="Predicted Value",
)

# Set the plot size
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

Before we move on, let's do a quick check to see how the polynomial features compare with what we know to be true for the real model!

We can get descriptive names for the features with `model[0].get_feature_names_out()` and we can get the fitted coefficients with `model[1].coef_`.

````{tip}
If you're ever confused about what methods/functions/attributes that are available on an object, try `dir(object)`! I use this all the time because I can't remember all of the methods for every type of model. 

Try it with `dir(model[0])`!
````

In [ ]:
for param, name in zip(model[1].coef_, model[0].get_feature_names_out()):
    print(f"The coefficient for {name} is {param:0.2f}")

Remember the real functional form was $f(x_0, x_1) = x_0^4+x_1^4+2x_0x_1^2+2x_1x_0^2-21x_0^2-13x_1^2-14x_0-22x_1+170$. Most of the coefficients are pretty good! There are a few that are close to zero but not quite. 


## Model capacity and under/over-fitting data

Let's see how the number of polynomial features affects the fit for both the train and validation data. 

With 60 samples, this actually looks pretty good! Let's see how our model works with varying model complexity.

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = generate_trainvaltest_data(60)

train_rmse = {}
val_rmse = {}
for degree in range(10):

    # Make and fit a model with the appropriate degree!
    model = LinearRegression()
    model = make_pipeline(
        PolynomialFeatures(degree), LinearRegression(fit_intercept=False)
    )
    model.fit(X_train, y_train)

    # Save the RMSE results in the dictionaries above
    train_rmse[degree] = mean_squared_error(y_train, model.predict(X_train)) ** 0.5
    val_rmse[degree] = mean_squared_error(y_val, model.predict(X_val)) ** 0.5

In [ ]:
# Plot the generalization gap!
fig = go.Figure(
    data=[
        go.Scatter(
            x=list(train_rmse.keys()), y=list(train_rmse.values()), name="Training"
        )
    ]
)
fig.add_scatter(x=list(val_rmse.keys()), y=list(val_rmse.values()), name="Validation")

fig.update_xaxes(
    title_text="Model Capacity (Polynomial Order)",
)
fig.update_yaxes(
    range=[0, 300],
    title_text="Model Fit (RMSE)",
)

# Set the plot size
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

There's a couple of interesting things here:
* The validation curve is usually above the training curve since the model hasn't seen that data before
* The training curve monotonically decreases with increasing capacity for these features and model
* The validation curve goes through a minimum at polynomial order 4, then stays roughly flat, then shoots up


````{note}
We call the gap between the training and validation results the **generalization gap**, since it shows how the models are able to generalize to the data in the validation set. Good features/models will have small generalization gaps, and by making smart modeling choices we can reduce it!
````

### Practice

Repeat the above with more than 60 data points. How do the results change?

## Reducing the generalization gap

To summarize the above:
* We can generate many features which may be useful in building a model
* Generating features that are actually used in the model can be very helpful
* If we generate too many features, we can actually make the model worse for predicting new data

We have a couple options:
* Tune the number or type of features being generated to minimize the validation error
* Change the model or features
* Change how we train the model (which we'll talk about here!)

### No regularization (linear regression)

As a reminder from the previous lecture on regression, we can think of linear regression as finding the parameters $\theta_i$ by minimizing the loss ($\mathcal{L}$) sum squared errors:
\begin{align*}
\min_{\theta}\mathcal{L} =\min_{\theta}  \left(y_i-\hat{y}_i\right)^2
\end{align*}
To convince ourselves that this is true, let's do a quick demonstration using minimize to fit the loss function explicitly and plot the coefficients. 

In [ ]:
from scipy.optimize import minimize


# Sum squared error loss function!
def loss(theta, X, y):
    y_pred = X @ theta
    loss = sum((y - y_pred) ** 2)
    return loss


# Linear regression fit using scipy.optimize.minimize
features = PolynomialFeatures(4).fit_transform(X_train)
sol = minimize(
    loss,
    np.ones(features.shape[1]),
    args=(features, y_train),
)
print(sol.x)

Now, let's do the same thing with scikit-learn. We should get the same results!

In [ ]:
# Linear Regression fit using sklearn
model = LinearRegression()
model = make_pipeline(PolynomialFeatures(4), LinearRegression(fit_intercept=False))
model.fit(X_train, y_train)

# Print the coefficients from sklearn, showing you get the same answer using either method!
print(model[1].coef_)

These coefficients are exactly the same as the custom model we optimized. As a final step and for comparison with the next steps, we'll repeat the plot from above to see how the train/validation errors scale with maximum polynomial degree.

In [ ]:
# Plot the generalization gap!
fig = go.Figure(
    data=[
        go.Scatter(
            x=list(train_rmse.keys()), y=list(train_rmse.values()), name="Training"
        )
    ]
)
fig.add_scatter(x=list(val_rmse.keys()), y=list(val_rmse.values()), name="Validation")

fig.update_xaxes(
    title_text="Model Capacity (Polynomial Order)",
)
fig.update_yaxes(
    range=[0, 300],
    title_text="Model Fit (RMSE)",
)

# Set the plot size
fig.update_layout(autosize=False, width=600, height=400)
fig.show()

### L2 regularization (ridge regression, or Tikhonov regularization)


Regularization refers to a small change in how we fit our models. We're not going to change the model at all - we're only going to train the fitting procedure. Specifically, we're going to add a term to our loss function that includes the absolute size of the fitted parameters. 

One of the most common types of regularization is L2 regularization. When this is applied to a linear model, it's called ridge regression. This approach is often effective at reducing the train/validation gap:
\begin{align*}
\min_{\theta}\mathcal{L} =\min_{\theta} \left[ \left(y_i-\hat{y}_i\right)^2 + \alpha \sum_j \theta_j^2\right]
\end{align*}
where $\theta_j$ are the model parameters, and $\alpha$ is a parameter that we can adjust to control the regularization.

`````{note}
* **Regularization:** Strategy to reduce the train/validation gap
* **L2 regularization:** Otherwise known as Tikhonov regularization, or Ridge regression if the model is linear
* $\alpha$: an empirical parameter that controls how strong the regularization is 
`````

`````{warning}
Large  $\alpha$ effectively reduce the model capacity. If you make the $\alpha$ too large it will hurt the training errors!
`````

To demonstrate this we'll again do the fit using a custom loss function, and using the `Ridge` model from sklearn.

In [ ]:
from scipy.optimize import minimize


# Sum squared error loss function!
def loss(theta, X, y, alpha=1):
    y_pred = X @ theta
    loss = sum((y - y_pred) ** 2) + alpha * (theta**2.0).sum()
    return loss


# Linear regression fit using scipy.optimize.minimize, showing you get the same answer using either method!
sol = minimize(
    loss,
    np.ones(model[1].coef_.shape),
    args=(PolynomialFeatures(4).fit_transform(X_train), y_train, 1),
)
print(sol.x)

In [ ]:
from sklearn.linear_model import Ridge

# Linear Regression fit using sklearn
model = LinearRegression()
model = make_pipeline(PolynomialFeatures(4), Ridge(fit_intercept=False))
model.fit(X_train, y_train)

# Print the coefficients from sklearn, showing you get the same answer using either method!
print(model[1].coef_)

Same coefficients either way! Let's see how this affects the model performance as the number of features increases.

In [ ]:
from sklearn.linear_model import Ridge

X_train, X_val, X_test, y_train, y_val, y_test = generate_trainvaltest_data(60)

train_rmse = {}
val_rmse = {}
for degree in range(10):

    # Make and fit a model with the appropriate degree!
    model = LinearRegression()
    model = make_pipeline(
        PolynomialFeatures(degree), Ridge(fit_intercept=False, alpha=1)
    )
    model.fit(X_train, y_train)

    # Save the RMSE results in the dictionaries above
    train_rmse[degree] = mean_squared_error(y_train, model.predict(X_train)) ** 0.5
    val_rmse[degree] = mean_squared_error(y_val, model.predict(X_val)) ** 0.5

In [ ]:
# Plot the generalization gap!
fig = go.Figure(
    data=[
        go.Scatter(
            x=list(train_rmse.keys()), y=list(train_rmse.values()), name="Training"
        )
    ]
)
fig.add_scatter(x=list(val_rmse.keys()), y=list(val_rmse.values()), name="Validation")

fig.update_xaxes(
    title_text="Model Capacity (Polynomial Order)",
)
fig.update_yaxes(
    range=[0, 300],
    title_text="Model Fit (RMSE)",
)

# Set the plot size
fig.update_layout(autosize=False, width=600, height=400)
fig.show()

### Practice: Vary the alpha parameter and describe the results

### L1 regularization (LASSO)

Another common regularization strategy is L1 regularization. When applied to linear models this is called LASSO regression.
\begin{align*}
\min_{\theta}\mathcal{L} =\min_{\theta} \left[ \frac{\left(y_i-\hat{y}_i\right)^2}{2N} + \alpha \sum_j |\theta_j|\right]
\end{align*}
where $\theta_j$ are the model parameters, and $\alpha$ is a parameter that we can adjust to control the regularization, and N is the number of training data. The LASSO model has the interesting property that, after minimization, many of the parameters tend to be 0. Non-zero elements will tell you which features are important. LASSO can be used as a feature selection strategy!

`````{note}
* **Feature selection:** A step in an ML process to identify which features are most important and only use those in the final model
* **LASSO:** L1 regularization applied to linear regression. LASSO tends to set some parameters equal to 0!
`````




In [ ]:
from scipy.optimize import minimize


# Sum squared error loss function!
def loss(theta, X, y, alpha=1):
    y_pred = X @ theta

    # Loss from https://github.com/scikit-learn/scikit-learn/blob/36958fb24/sklearn/linear_model/_coordinate_descent.py#L1134
    loss = sum((y - y_pred) ** 2) / 2 / len(y) + alpha * np.abs(theta).sum()
    return loss


# Linear regression fit using scipy.optimize.minimize, showing you get the same answer using either method!
features = PolynomialFeatures(4).fit_transform(X_train)
sol = minimize(
    loss,
    np.zeros(features.shape[1]),
    args=(features, y_train, 1),
)
print(sol.x)

In [ ]:
from sklearn.linear_model import Lasso

# Linear Regression fit using sklearn
model = Lasso()
model = make_pipeline(PolynomialFeatures(4), Lasso(fit_intercept=False))
model.fit(X_train, y_train)

# Print the coefficients from sklearn, showing you get the same answer using either method!
print(model[1].coef_)

In [ ]:
from sklearn.linear_model import Lasso

X_train, X_val, X_test, y_train, y_val, y_test = generate_trainvaltest_data(60)

train_rmse = {}
val_rmse = {}
for degree in range(14):

    # Make and fit a model with the appropriate degree!
    model = LinearRegression()
    model = make_pipeline(
        PolynomialFeatures(degree),
        Lasso(fit_intercept=False, alpha=0.1, max_iter=100000),
    )
    model.fit(X_train, y_train)

    # Save the RMSE results in the dictionaries above
    train_rmse[degree] = mean_squared_error(y_train, model.predict(X_train)) ** 0.5
    val_rmse[degree] = mean_squared_error(y_val, model.predict(X_val)) ** 0.5

In [ ]:
# Plot the generalization gap!
fig = go.Figure(
    data=[
        go.Scatter(
            x=list(train_rmse.keys()), y=list(train_rmse.values()), name="Training"
        )
    ]
)
fig.add_scatter(x=list(val_rmse.keys()), y=list(val_rmse.values()), name="Validation")

fig.update_xaxes(
    title_text="Model Capacity (Polynomial Order)",
)
fig.update_yaxes(
    range=[0, 300],
    title_text="Model Fit (RMSE)",
)

# Set the plot size
fig.update_layout(autosize=False, width=600, height=400)
fig.show()

Notice the validation error, especially with a very large model, is significantly better here. There's some variability depending on how the model is fit and the initial parameters used before the minimization. 

In [ ]:
model[1].coef_

As one final check, let's see how the results for parameter coefficients compare with the known Himmelblau function parameters from above!
\begin{align}
y=f(x_0, x_1) &= (x_0^2+x_2-11)^2+(x_0+x_1^2-7)^2\\
&=x_0^4+x_1^4+2x_0x_1^2+2x_1x_0^2-21x_0^2-13x_1^2-14x_0-22x_1+170
\end{align}

In [ ]:
for param, name in zip(model[-1].coef_, model[0].get_feature_names_out()):
    if np.abs(param) > 0.01:
        print(f"The coefficient for {name} is {param:0.2f}")

### Practice: Vary the alpha parameter and describe the results

`````{note}
Summary!
* We can add many possible features to our ML models
* We should be careful about how our model performs for train/validation as we increase the model complexity or increase the number of features
* We can reduce the gap between train and validation errors by adding parameter regularization to the loss function
* LASSO can be used to identify important features by forcing unimportant feature weights to zero!
`````